In [1]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup
import json
from io import StringIO
import datetime
from google.cloud import bigquery
from google.oauth2 import service_account

# Import data using BeautifulSoup
url = 'https://en.vedur.is/earthquakes-and-volcanism/earthquakes/#view=table'
# Create object page
page = requests.get(url)
soup = BeautifulSoup(page.text, 'lxml')
table1 = soup.find_all('script')
table2 = str(table1[7]).splitlines()
data_str = table2[10].replace("VI.quakeInfo = ", "").replace(";","")
data_str = re.sub(r"new Date\((\d+),(\d+)-1,(\d+),(\d+),(\d+),(\d+)\)", r"'\1-\2-\3T\4:\5:\6'", data_str)
data_str = data_str.replace("'", '"')

# convert it to Json object
data_json = json.loads(data_str)

# Details for BigQuery client
credentials_path = 'big-data-final-project-407314-840759d2dedd.json'
dataset_id = 'earthquakes'
table_id = 'quakes'
project_id = 'big-data-final-project-407314'

# Initialize BigQuery client
credentials = service_account.Credentials.from_service_account_file(credentials_path)
client = bigquery.Client(credentials=credentials, project=project_id)
table_ref = f"{project_id}.{dataset_id}.{table_id}"

# filtering json data to select only day-1 seismic activity 
# To prevent duplicate data we set a process in place to injest every today the (today-1) data
def filter_json_data(json_array, time_key):
    filtered_data = []
    yesterday_start = datetime.datetime.now() - datetime.timedelta(days=1)
    yesterday_start = yesterday_start.replace(hour=0, minute=0, second=0, microsecond=0)
    today_start = datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)

    for item in json_array:
        try:
            # Parse the datetime using strptime
            item_datetime = datetime.datetime.strptime(item[time_key], '%Y-%m-%dT%H:%M:%S')
            # Check if the item's datetime is within the desired range
            if yesterday_start <= item_datetime < today_start:
                filtered_data.append(item)
        except ValueError as e:
            # Handle the error or log it if necessary
            print(f"Error parsing date: {e}")

    return filtered_data

# filter data for specific timeframe
filtered_data = filter_json_data(data_json, 't')
for item in filtered_data:
    errors = client.insert_rows_json(table_ref, [item])
    if errors:
        print("Encountered errors while inserting rows:", errors)
    else:
        print("Row inserted successfully.")

Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
Row inserted successfully.
R